In [ ]:
import json
import os
from research.utils.data_access_utils import S3AccessUtils
from matplotlib import pyplot as plt

In [ ]:
pen_id = 164

weight_distribution = {
    '0.0-1.0': 0.0,
    '1.0-2.0': 1.05,
    '2.0-3.0': 19.52,
    '3.0-4.0': 64.46,
    '4.0-5.0': 14.6,
    '5.0-6.0': 0.35,
    '6.0-7.0': 0.024,
    '7.0-8.0': 0.0,
    '8.0-9.0': 0.0
}

gutted_average_weight = 3440
expected_loss_factor = 16.0


ground_truth_metadata = {
    'pen_id': pen_id,
    'gutted_weight_distribution': weight_distribution,
    'gutted_average_weight': gutted_average_weight,
    'expected_loss_factor': expected_loss_factor,
    'last_feeding_date': '2020-10-01',
    'undeployment_date': '2020-10-06',
    'harvest_date': None,
    'slaughter_date': None
}

In [ ]:
pen_id = 100
weight_distribution = None
gutted_average_weight = 4508
expected_loss_factor = 17.5


ground_truth_metadata = {
    'pen_id': pen_id,
    'gutted_weight_distribution': weight_distribution,
    'gutted_average_weight': gutted_average_weight,
    'expected_loss_factor': expected_loss_factor,
    'last_feeding_date': '2020-08-24',
    'undeployment_date': '2020-08-28',
    'harvest_date': '2020-08-29',
    'slaughter_date': '2020-08-29'
}

In [ ]:
ground_truth_metadata



In [ ]:
json.dump(ground_truth_metadata, open('/root/data/alok/biomass_estimation/playground/ground_truth_metadata.json', 'w'))



In [ ]:
f = '/root/data/alok/biomass_estimation/playground/ground_truth_metadata.json'



In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))



In [ ]:
s3.s3_client.upload_file(f, 'aquabyte-images-adhoc', 'alok/production_datasets/eldviktaren_pen_id_164_2020-09-06_2020-10-06/ground_truth_metadata.json')



In [ ]:
df[(df.weight >= 3) & (df.weight <= 4)].shape[0] / df.shape[0]

In [ ]:
import pandas as pd

df = pd.read_csv('/root/data/alok/biomass_estimation/playground/blom_ground_truth_data.csv', header=None)

In [ ]:
df.columns = ['a', 'site', 'b', 'c', 'grade', 'weight', 'measured_at']

In [ ]:
weights = 1000 * df.weight.values / .83

In [ ]:
import numpy as np
bucket_cutoffs = np.arange(0, 10000, 1000)

In [ ]:
def get_distribution(weights, bucket_cutoffs):
    dist = {}
    count = 0
    for low, high in zip(bucket_cutoffs, bucket_cutoffs[1:]):
        bucket = f'{1e-3 * low}-{1e-3 * high}'
        bucket_count = weights[(weights >= low) & (weights < high)].shape[0]
        dist[bucket] = bucket_count
        count += bucket_count
    
    dist = {k: round(100 * v / count, 1) for k, v in dist.items()}
    return dist

In [ ]:
get_distribution(weights, bucket_cutoffs)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(weights)
plt.grid()
plt.show()